# 🏥 Nursing FunctionGemma Demo

This notebook demonstrates the **Nursing FunctionGemma** model's ability to convert natural language into structured function calls.

**Available Tools:**
- `record_vitals(systolic, diastolic, heart_rate, temp_c)` - Record patient vitals
- `administer_medication(drug_name, dose, route)` - Log medication administration
- `search_nmc_standards(query)` - Search NMC guidance

In [ ]:
!pip install -q -U transformers peft torch accelerate bitsandbytes huggingface_hub

In [ ]:
from huggingface_hub import login
login()

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

# Model IDs
BASE_MODEL_ID = "google/medgemma-4b-it"
ADAPTER_ID = "NurseCitizenDeveloper/nursing-function-gemma"

print("Loading base model...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

print(f"Loading FunctionGemma adapter from: {ADAPTER_ID}")
model = PeftModel.from_pretrained(model, ADAPTER_ID)
print("✅ Model loaded!")

In [ ]:
def call_function_gemma(user_input):
    """Send a request to FunctionGemma and get the function call output."""
    
    tools_prompt = """You are a clinical AI agent. You have access to the following tools:
- record_vitals(systolic, diastolic, heart_rate, temp_c)
- administer_medication(drug_name, dose, route)
- search_nmc_standards(query)
If the user's request requires a tool, output the function call XML."""

    prompt = f"<start_of_turn>user\n{tools_prompt}\n\n{user_input}<end_of_turn>\n<start_of_turn>model\n"
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=False,  # Deterministic for function calls
            temperature=0.1,
            eos_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract just the model's response
    if "<start_of_turn>model" in response:
        response = response.split("<start_of_turn>model")[-1].strip()
    
    return response

## 🧪 Test Cases

Let's test the model with different nursing scenarios!

In [ ]:
# Test 1: Record Vitals
print("=" * 50)
print("TEST 1: Recording Vitals")
print("=" * 50)
user_input = "Patient's BP is 120/80, pulse 72, temp 37.2"
print(f"Input: {user_input}")
print(f"Output: {call_function_gemma(user_input)}")

In [ ]:
# Test 2: Medication Administration
print("=" * 50)
print("TEST 2: Medication Administration")
print("=" * 50)
user_input = "I've given Paracetamol 1g orally"
print(f"Input: {user_input}")
print(f"Output: {call_function_gemma(user_input)}")

In [ ]:
# Test 3: NMC Standards Search
print("=" * 50)
print("TEST 3: NMC Standards Search")
print("=" * 50)
user_input = "What does the NMC say about confidentiality?"
print(f"Input: {user_input}")
print(f"Output: {call_function_gemma(user_input)}")

In [ ]:
# Test 4: IV Medication
print("=" * 50)
print("TEST 4: IV Medication")
print("=" * 50)
user_input = "Administered Morphine 10mg IV"
print(f"Input: {user_input}")
print(f"Output: {call_function_gemma(user_input)}")

## 🎯 Interactive Demo

Try your own inputs below!

In [ ]:
# Interactive: Enter your own input
your_input = "Just took obs: Heart rate 88, Blood pressure 140 over 90, temp 38.1"
print(f"Your Input: {your_input}")
print(f"\nFunctionGemma Output:\n{call_function_gemma(your_input)}")

---

## ✅ Success Criteria

The model should output structured function calls like:
```
<function_call>record_vitals(systolic=120, diastolic=80, heart_rate=72, temp_c=37.2)</function_call>
```

or

```
<function_call>administer_medication(drug_name='Paracetamol', dose='1g', route='PO')</function_call>
```

---

**Developed by Lincoln Gombedza | Nursing Citizen Development** 🩺